# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 41 new papers today
          18 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/17 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2301.00010


/tmp/ipykernel_2098/4030337529.py:34: LatexWarning: 2301.00010 did not run properly
not a gzip file
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


Retrieving document from  https://arxiv.org/e-print/2301.00017


extracting tarball to tmp_2301.00017...

...

 done.
Retrieving document from  https://arxiv.org/e-print/2301.00705


extracting tarball to tmp_2301.00705...

 done.
Retrieving document from  https://arxiv.org/e-print/2301.00741


extracting tarball to tmp_2301.00741...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.00578-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.00578) | **PHANGS-JWST First Results: Variations in PAH Fraction as a Function of  ISM Phase and Metallicity**  |
|| Jérémy Chastenet, et al. -- incl., <mark>Daizhong Liu</mark>, <mark>Frank Bigiel</mark>, <mark>Annie Hughes</mark>, <mark>Kathryn Kreckel</mark>, <mark>Eva Schinnerer</mark> |
|*Appeared on*| *2023-01-03*|
|*Comments*| *13 pages, 4 figures. Accepted as part of a PHANGS-JWST Focus Issue to appear in ApJ*|
|**Abstract**| We present maps tracing the fraction of dust in the form of polycyclic aromatic hydrocarbons (PAHs) in IC 5332, NGC 628, NGC 1365, and NGC 7496 from JWST/MIRI observations. We trace the PAH fraction by combining the F770W ($7.7~\mu$m) and F1130W ($11.3~\mu$m) filters to track ionized and neutral PAH emission, respectively, and comparing the PAH emission to F2100W which traces small, hot dust grains. We find average $R{\rm_{PAH} = (F770W+F1130W)/F2100W}$ values of 3.3, 4.7, 5.1, and 3.6 in IC 5332, NGC 628, NGC 1365, and NGC 7496, respectively. We find that H II regions traced by MUSE H$\alpha$ show a systematically low PAH fraction. The PAH fraction remains relatively constant across other galactic environments, with slight variations. We use CO + H I + H$\alpha$ to trace the interstellar gas phase and find that the PAH fraction decreases above a value of I$_{H\alpha}/\Sigma_{H~I+H_2}$ $\sim~10^{37.5}$ erg s$^{-1}$ kpc$^{-2}$ (M$_\odot$ pc$^{-2}$)$^{-1}$, in all four galaxies. Radial profiles also show a decreasing PAH fraction with increasing radius, correlated with lower metallicity, in line with previous results showing a strong metallicity dependence to the PAH fraction. Our results suggest that the process of PAH destruction in ionized gas operates similarly across the four targets. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.00017-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.00017) | **Broad emission lines in optical spectra of hot dust-obscured galaxies  can contribute significantly to JWST/NIRCam photometry**  |
|| Jed McKinney, et al. -- incl., <mark>Lee Armus</mark>, <mark>Keith Matthews</mark> |
|*Appeared on*| *2023-01-03*|
|*Comments*| *8 pages, 3 figures, 1 table, submitted to ApJL*|
|**Abstract**| Selecting the first galaxies at z>7-10 from JWST surveys is complicated by z<6 contaminants with degenerate photometry. For example, strong optical nebular emission lines at z<6 may mimic JWST/NIRCam photometry of z>7-10 Lyman Break Galaxies (LBGs). Dust-obscured 3<z<6 galaxies in particular are potentially important contaminants, and their faint rest-optical spectra have been historically difficult to observe. A lack of optical emission line and continuum measures for 3<z<6 dusty galaxies now makes it difficult to test their expected JWST/NIRCam photometry for degenerate solutions with NIRCam dropouts. Towards this end, we quantify the contribution by strong emission lines to NIRCam photometry in a physically motivated manner by stacking 21 Keck II/NIRES spectra of hot, dust-obscured, massive ($\log\mathrm{M_*/M_\odot}\gtrsim10-11$) and infrared (IR) luminous galaxies at z~1-4. We derive an average spectrum and measure strong narrow (broad) [OIII]5007 and H$\alpha$ features with equivalent widths of $130\pm20$ A ($150\pm50$ A) and $220\pm30$ A ($540\pm80$ A) respectively. These features can increase broadband NIRCam fluxes by factors of 1.2-1.7 (0.2-0.6 mag). Due to significant dust-attenuation ($A_V\sim6$), we find H$\alpha$+[NII] to be significantly brighter than [OIII]+H$\beta$, and therefore find that emission-line dominated contaminants of high-z galaxy searches can only reproduce moderately blue perceived UV continua of $S_\lambda\propto\lambda^\beta$ with $\beta>-1.5$ and z>4. While there are some redshifts (z~3.75) where our stack is more degenerate with the photometry of z>10 LBGs between $\lambda_{rest}\sim0.3-0.8\,\mu$m, redder filter coverage beyond $\lambda_{obs}>3.5\,\mu$m and far-IR/sub-mm follow-up may be useful for breaking the degeneracy and making a crucial separation between two fairly unconstrained populations, dust-obscured galaxies at z~3-6 and LBGs at z>10. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.00022-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.00022) | **Bubble in the Whale: Identifying the Optical Counterparts and Extended  Nebula for the Ultraluminous X-ray Sources in NGC 4631**  |
|| <mark>Jing Guo</mark>, et al. -- incl., <mark>Shiwu Zhang</mark>, <mark>Jifeng Liu</mark> |
|*Appeared on*| *2023-01-03*|
|*Comments*| *17 pages, 10 figures, accepted by ApJ*|
|**Abstract**| We present a deep optical imaging campaign on the starburst galaxy NGC 4631 with CFHT/MegaCam. By supplementing the HST/ACS and Chandra/ACIS archival data, we search for the optical counterpart candidates of the five brightest X-ray sources in this galaxy, four of which are identified as ultraluminous X-ray sources (ULXs). The stellar environments of the X-ray sources are analyzed using the extinction-corrected color-magnitude diagrams and the isochrone models. We discover a highly asymmetric bubble nebula around X4 which exhibits different morphology in the H$\alpha$ and [O III] images. The [O III]/H$\alpha$ ratio map shows that the H$\alpha$-bright bubble may be formed mainly via the shock ionization by the one-sided jet/outflow, while the more compact [O III] structure is photoionized by the ULX. We constrain the bubble expansion velocity and interstellar medium density with the MAPPINGS V code, and hence estimate the mechanical power injected to the bubble as $P_w \sim 5\times10^{40}$ erg s$^{-1}$ and the corresponding bubble age of $\sim7\times 10^{5}$ yr. Relativistic jets are needed to provide such level of mechanical power with a mass-loss rate of $\sim10^{-7}\ M_{\odot}\ \rm yr^{-1}$. Besides the accretion, the black hole spin is likely an additional energy source for the super-Eddington jet power. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.00705-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.00705) | **White Dwarfs with Infrared Excess from LAMOST Data Release 5**  |
|| <mark>Lin Wang</mark>, et al. -- incl., <mark>Xiaoxia Zhang</mark>, <mark>Zhi-Xiang Zhang</mark>, <mark>Jincheng Guo</mark> |
|*Appeared on*| *2023-01-03*|
|*Comments*| *15 pages, 7 figures, 3 tables; accepted for publication in ApJ*|
|**Abstract**| Infrared excess is an important probe of sub-stellar companions and/or debris disks around white dwarfs (WDs). Such systems are still rare for in-depth understanding of their formation and long-term evolution. One of the largest spectroscopic surveys carried out by the Large sky Area Multi-Object fiber Spectroscopic Telescope (LAMOST) recently released more than $3000$ WDs, a significant fraction of which have not undergone excess search. Here we present cross-correlation of LAMOST DR5 WD catalog with the Pan-STARRS, SDSS, UKIDSS, 2MASS, and {\it WISE}. By performing SED (spectral energy distribution) fitting for 846 WDs with $WISE$ detections, we identify 50 candidates with infrared excess, including 7 candidate WD+M dwarf binaries, 31 candidate WD+brown dwarf (BD) binaries and 12 candidate WD+dust disk systems. 8 of the dust disk systems are our new identifications. Utilizing a systematic survey with accurate stellar parameters derived from spectral fitting, our work is an important addition to previous searches for infrared excess from SDSS and {\it Gaia} WDs, and provides a significant ($\gtrsim8\%$) complement to current database of WDs with candidate BD companions and dust disks. The frequencies of WD+BD binaries and WD+dust disk systems are constrained to be $\lesssim3.7\%$ and $\sim1.4\%$, respectively. The properties of candidate dust disk systems are discussed. All of our candidates require follow-up observations for confirmation owing to limited spatial resolution of {\it WISE}. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.00741-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.00741) | **Analysis of the first infrared spectrum of quasi-bound H2 line emission  in Herbig-Haro 7**  |
|| Evelyne Roueff, Michael G. Burton, Thomas R. Geballe, <mark>Hervé Abgrall</mark> |
|*Appeared on*| *2023-01-03*|
|*Comments*| *5 pages, 3 figures, accepted in Astronomy and Astrophysics Letters 22/12/2022*|
|**Abstract**| Context. Highly excited molecular hydrogen H2 has been observed in many regions of shocked molecular gas. A recently published $K$-band spectrum of Herbig-Haro 7 (HH7) contains several vibration-rotation lines of H2 from highly excited energy levels that have not been detected elsewhere, including a line at 2.179 $\mu$m identified as arising from the $v$=2 $J$=29 level, which lies above the dissociation limit of H2. One emission line at 2.104 $\mu$m in this spectrum was unidentified. Aims. We aim to complete the analysis of the spectrum of HH7 by including previously missing molecular data that have been recently computed. Methods. We re-analysed the $K$-band spectrum, emphasising the physics of quasi-bound upper levels that can produce infrared emission lines in the $K$ band. Results. We confirm the identification of the $2-1$ $S$(27) line at 2.1785 $\mu$m and identify the line at 2.1042 $\mu$m as due to the 1-0 $S$(29) transition of H2, whose upper level energy is also higher than the dissociation limit. This latter identification, its column density, and the energy of its upper level further substantiate the existence of a hot thermal component at 5000 K in the HH7 environment.} Conclusion. The presence of the newly identified $1-0$ $S$(29) line, whose quasi-bound upper level ($v$=1, $J$=31) has a significant spontaneous dissociation probability, shows that dissociation of H2 is occurring. The mechanism by which virtually all of the H2 levels with energies from 20,000 K to 53,000 K is maintained in local thermodynamic equilibrium at a single temperature of $\sim$5,000 K remains to be understood. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.00113-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.00113) | **Black hole images: A Review**  |
|| Songbai Chen, Jiliang Jing, <mark>Wei-Liang Qian</mark>, Bin Wang |
|*Appeared on*| *2023-01-03*|
|*Comments*| *17 pages, 8 figures, Acepted by Sci. China-Phys. Mech. Astron*|
|**Abstract**| In recent years, unprecedented progress has been achieved regarding black holes' observation through the electromagnetic channel. The images of the supermassive black holes M87$^{*}$ and Sgr A$^{*}$ released by the Event Horizon Telescope (EHT) Collaboration provided direct visual evidence for their existence, which has stimulated further studies on various aspects of the compact celestial objects. Moreover, the information stored in these images provides a new way to understand the pertinent physical processes that occurred near the black holes, to test alternative theories of gravity, and to furnish insight into fundamental physics. In this review, we briefly summarize the recent developments on the topic. In particular, we elaborate on the features and formation mechanism of black hole shadows, the properties of black hole images illuminated by the surrounding thin accretion disk, and the corresponding polarization patterns. The potential applications of the relevant studies are also addressed. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error 'utf-8' codec can't decode byte 0xa8 in position 2991: invalid start byte</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.00010-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.00010) | **Exploring the Solar Poles: The Last Great Frontier of the Sun**  |
|| Dibyendu Nandy, et al. -- incl., <mark>Sushant S. Mahajan</mark> |
|*Appeared on*| *2023-01-03*|
|*Comments*| *This White Paper was submitted in 2022 to the United States National Academies Solar and Space Physics (Heliophysics) Decadal Survey*|
|**Abstract**| Despite investments in multiple space and ground-based solar observatories by the global community, the Sun's polar regions remain unchartered territory - the last great frontier for solar observations. Breaching this frontier is fundamental to understanding the solar cycle - the ultimate driver of short-to-long term solar activity that encompasses space weather and space climate. Magnetohydrodynamic dynamo models and empirically observed relationships have established that the polar field is the primary determinant of the future solar cycle amplitude. Models of solar surface evolution of tilted active regions indicate that the mid to high latitude surges of magnetic flux govern dynamics leading to the reversal and build-up of polar fields. Our theoretical understanding and numerical models of this high latitude magnetic field dynamics and plasma flows - that are a critical component of the sunspot cycle - lack precise observational constraints. This limitation compromises our ability to observe the enigmatic kilo Gauss polar flux patches and constrain the polar field distribution at high latitudes. The lack of these observations handicap our understanding of how high latitude magnetic fields power polar jets, plumes, and the fast solar wind that extend to the boundaries of the heliosphere and modulate solar open flux and cosmic ray flux within the solar system. Accurate observation of the Sun's polar regions, therefore, is the single most outstanding challenge that confronts Heliophysics. This paper argues the scientific case for novel out of ecliptic observations of the Sun's polar regions, in conjunction with existing, or future multi-vantage point heliospheric observatories. Such a mission concept can revolutionize the field of Heliophysics like no other mission concept has - with relevance that transcends spatial regimes from the solar interior to the heliosphere. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.00373-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.00373) | **Photometric IGM tomography with Subaru/HSC: the large-scale structure of  Ly$α$ emitters and IGM transmission in the COSMOS field at $z\sim5$**  |
|| Koki Kakiichi, et al. -- incl., <mark>Joseph F. Hennawi</mark>, <mark>Romain A. Meyer</mark>, <mark>Sarah I. Bosman</mark> |
|*Appeared on*| *2023-01-03*|
|*Comments*| *27 pages, 21 figures, submitted to MNRAS, table available as online supplementary material*|
|**Abstract**| We present a novel technique called "photometric IGM tomography" to map the intergalactic medium (IGM) at $z\simeq4.9$ in the COSMOS field. It utilizes deep narrow-band (NB) imaging to photometrically detect faint Ly$\alpha$ forest transmission in background galaxies across the Subaru/Hyper-Suprime Cam (HSC)'s $1.8\rm\,sq.\,deg$ field of view and locate Ly$\alpha$ emitters (LAEs) in the same cosmic volume. Using ultra-deep HSC images and Bayesian spectral energy distribution fitting, we measure the Ly$\alpha$ forest transmission at $z\simeq4.9$ along a large number ($140$) of background galaxies selected from the DEIMOS10k spectroscopic catalogue at $4.98<z<5.89$ and the SILVERRUSH LAEs at $z\simeq5.7$. We photometrically measure the mean Ly$\alpha$ forest transmission and achieve a result consistent with previous measurements based on quasar spectra. We also measure the angular LAE-Ly$\alpha$ forest cross-correlation and Ly$\alpha$ forest auto-correlation functions and place an observational constraint on the large-scale fluctuations of the IGM around LAEs at $z\simeq4.9$. Finally, we present the reconstructed 2D tomographic map of the IGM, co-spatial with the large-scale structure of LAEs, at a transverse resolution of $11 \,h^{-1}\rm cMpc$ across $140\,h^{-1}\rm cMpc$ in the COSMOS field at $z\simeq4.9$. We discuss the observational requirements and the potential applications of this new technique for understanding the sources of reionization, quasar radiative history, and galaxy-IGM correlations across $z\sim3-6$. Our results represent the first proof-of-concept of photometric IGM tomography, offering a new route to examining early galaxy evolution in the context of the large-scale cosmic web from the epoch of reionization to cosmic noon. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error sequence item 0: expected str instance, TexNode found</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2301.00578.md
    + _build/html/tmp_2301.00578/./ratio_abdance_panels_2.png
    + _build/html/tmp_2301.00578/./ratio_abdance_all_maps_panel_sat.png
    + _build/html/tmp_2301.00578/./ratio_abdance_hagas_2d_v2_2.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\XX}{\textbf{Flag!}\xspace}$
$\newcommand{\coto}{^{12}{\rm CO}~(2-1)\xspace}$
$\newcommand{\halpha}{H\alpha\xspace}$
$\newcommand{\hii}{\ion{H}{2}\xspace}$
$\newcommand{\hone}{H{\sc i}\xspace}$
$\newcommand{\hagasratio}{I_{\rm H{\alpha}}/\Sigma_{\rm H{\sc I}+H_2}\xspace}$
$\newcommand{\ratio}{(F770W+F1130W)/F2100W\xspace}$
$\newcommand{\rpah}{R_{\rm PAH}\xspace}$
$\newcommand{\mjysr}{MJy~sr^{-1}\xspace}$
$\newcommand{\JS}[1]{\textcolor{violet}{#1}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\XX$}{\textbf{Flag!}$\xspace$}$
$\newcommand{$\coto$}{^{12}{\rm CO}~(2-1)$\xspace$}$
$\newcommand{$\halpha$}{H\alpha$\xspace$}$
$\newcommand{$\hii$}{$\ion${H}{2}$\xspace$}$
$\newcommand{$\hone$}{H{\sc i}$\xspace$}$
$\newcommand{$\hagasratio$}{I_{\rm H{\alpha}}/\Sigma_{\rm H{\sc I}+H_2}$\xspace$}$
$\newcommand{$\ratio$}{(F770W+F1130W)/F2100W$\xspace$}$
$\newcommand{$\rpah$}{R_{\rm PAH}$\xspace$}$
$\newcommand{$\mjysr$}{MJy~sr^{-1}$\xspace$}$
$\newcommand{$\JS$}[1]{\textcolor{violet}{#1}}$</div>



<div id="title">

# PHANGS-JWST First Results: Variations in PAH Fraction as a Function of ISM Phase and Metallicity

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2301.00578-b31b1b.svg)](https://arxiv.org/abs/2301.00578)<mark>Appeared on: 2023-01-03</mark> - _13 pages, 4 figures. Accepted as part of a PHANGS-JWST Focus Issue to appear in ApJ_

</div>
<div id="authors">

J\'er\'emy Chastenet, et al. -- incl., <mark>Daizhong Liu</mark>, <mark>Frank Bigiel</mark>, <mark>Annie~Hughes</mark>, <mark>Kathryn Kreckel</mark>, <mark>Eva Schinnerer</mark>

</div>
<div id="abstract">

**Abstract:** We present maps tracing the fraction of dust in the form of polycyclic aromatic hydrocarbons (PAHs) in IC 5332, NGC 628, NGC 1365, and NGC 7496 from JWST/MIRI observations.We trace the PAH fraction by combining the F770W ($7.7 \mu$m) and F1130W ($11.3 \mu$m) filters to track ionized and neutral PAH emission, respectively, and comparing the PAH emission to F2100W which traces small, hot dust grains. We find average$R{\rm_{PAH} = (F770W+F1130W)/F2100W}$values of 3.3, 4.7, 5.1, and 3.6 in IC 5332, NGC 628, NGC 1365, and NGC 7496, respectively.We find that$\hii$regions traced by MUSE$\halpha$show a systematically low PAH fraction. The PAH fraction remains relatively constant across other galactic environments, with slight variations. We use CO+$\hone$+$\halpha$to trace the interstellar gas phase and find that the PAH fraction decreases above a value of${\rm I_{H\alpha}/\Sigma_{H{\sc I}+H_2}} \sim 10^{37.5} {\rm erg s^{-1} kpc^{-2} (M_\odot pc^{-2}})^{-1}$, in all four galaxies.  Radial profiles also show a decreasing PAH fraction with increasing radius, correlated with lower metallicity, in line with previous results showing a strong metallicity dependence to the PAH fraction. Our results suggest that the process of PAH destruction in ionized gas operates similarly across the four targets.

</div>

<div id="div_fig1">

<img src="tmp_2301.00578/./ratio_abdance_panels_2.png" alt="Fig2" width="100%"/>

**Figure 2. -** Running medians of $\rpah$, as a function of
    (\textit{top left:}) $r/{\rm r_{25}}$;
    (\textit{top right:}) ${\rm 12+log(O/H)}$ using metallicity maps from \citet[][]{Williams2022};
    (\textit{middle left:}) $\hagasratio$ in units of ${\rm  erg s^{-1} kpc^{-2} (M_\odot pc^{-2}})^{-1}$;
    (\textit{middle right:}) the fraction of molecular gas;
    (\textit{bottom left:}) the environmental masks from \citet[][]{Querejeta2021}, with a black star symbol showing the median for all pixels within each category.
    The error-bars show $3 \times$ standard error of the mean in each bin (except for the bottom left panel, only 1 SEM).
    Note that the middle panels involve the flat $\hone$ distribution assumption in IC 5332 and NGC 1365, which may shift the curves horizontally. (*FigMediansPanel*)

</div>
<div id="div_fig2">

<img src="tmp_2301.00578/./ratio_abdance_all_maps_panel_sat.png" alt="Fig1" width="100%"/>

**Figure 1. -** Maps of $\rpah$ in IC 5332 (top left), NGC 628 (top right), NGC 1365 (bottom left), and NGC 7496 (bottom right). We mask the pixels with a ${\rm S/N < 3}$ in all bands (gray uniform background).
    We also mask the central pixels in NGC 1365 and NGC 7496 which are saturated, using instrument PSFs, and perform a by-hand additional masking to remove conspicuous saturation artifacts (shown in gray scale, not included in the analysis).
    We plot contours for a few of the brightest $\hii$ regions. They are clearly visible as depressions (darker colors) in $\rpah$, especially in NGC 628 and NGC 7496. (*FigAllMaps*)

</div>
<div id="div_fig3">

<img src="tmp_2301.00578/./ratio_abdance_hagas_2d_v2_2.png" alt="Fig3" width="100%"/>

**Figure 3. -** 2D histograms of the $\hagasratio$ and the $\rpah$, color-coded by metallicity, using the gradient from PHANGS-MUSE.
    The more transparent colors indicate bins with at least 10 hits, while the solid colors with at least 100 hits per bin. (*FigHaGas_Ratio2D*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

149  publications in the last 7 days.
	 _build/html/2301.00578.md
	 _build/html/2212.12039.md
	 _build/html/2212.11991.md
	 _build/html/2212.11425.md
	 _build/html/2212.11177.md
	 _build/html/2212.10804.md
	 _build/html/2212.10512.md
	 _build/html/2212.09766.md
	 _build/html/2212.09675.md
	 _build/html/2212.09661.md
	 _build/html/2212.09652.md
	 _build/html/2212.09443.md
	 _build/html/2212.09307.md
	 _build/html/2212.09168.md
	 _build/html/2212.09159.md
	 _build/html/2212.08702.md
	 _build/html/2212.07332.md
	 _build/html/2212.06434.md
	 _build/html/2212.04710.md
	 _build/html/2212.04026.md
	 _build/html/2212.03563.md
	 _build/html/2212.03514.md
	 _build/html/2212.02667.md
	 _build/html/2212.02627.md
	 _build/html/2212.02235.md
	 _build/html/2212.01707.md
	 _build/html/2212.01526.md
	 _build/html/2212.01397.md
	 _build/html/2212.01291.md
	 _build/html/2212.01281.md
	 _build/html/2212.00812.md
	 _build/html/2212.00811.md
	 _build/html/2212.00806.md
	 _build/html/2212.00047.md
	 _build/h

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers